#Reference
Help taken from these sites:


1.    https://towardsdatascience.com/transfer-learning-with-vgg16-and-keras-50ea161580b4
2.   https://keras.io/api/applications/vgg/



In [ ]:
import os
import cv2
from PIL import Image
import tensorflow as tf
from keras import backend as K
from keras.models import load_model
#from keras.preprocessing.image import img_to_array
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator



In [ ]:
IMG_SHAPE  = 224
batch_size = 8


In [ ]:
train_dir = "/content/drive/MyDrive/Project/dataset-split/train"
test_dir = "/content/drive/MyDrive/Project/dataset-split/test"
valid_dir = "/content/drive/MyDrive/Project/dataset-split/val"

In [ ]:
image_gen_train = ImageDataGenerator(rescale = 1./255)
train_data_gen = image_gen_train.flow_from_directory(batch_size = batch_size,
directory = train_dir,
shuffle= True,
target_size = (IMG_SHAPE,IMG_SHAPE),
class_mode = 'binary')
image_generator_validation = ImageDataGenerator(rescale=1./255)
val_data_gen = image_generator_validation.flow_from_directory(batch_size=batch_size,
directory=valid_dir,
target_size=(IMG_SHAPE, IMG_SHAPE),
class_mode='binary')
image_gen_test = ImageDataGenerator(rescale=1./255)
test_data_gen = image_gen_test.flow_from_directory(batch_size=batch_size,
directory=test_dir,
target_size=(IMG_SHAPE, IMG_SHAPE),
class_mode='binary')
total_train=825
total_val=235

Found 1369 images belonging to 2 classes.
Found 391 images belonging to 2 classes.
Found 196 images belonging to 2 classes.


In [ ]:
from keras.applications import vgg16


img_rows, img_cols = 224, 224


pre_trained_model = vgg16.VGG16(weights = 'imagenet',
                 include_top = False,
                 input_shape = (img_rows, img_cols, 3))

# Here we freeze the last 4 layers
# Layers are set to trainable as True by default
for layer in pre_trained_model.layers:
    layer.trainable = False

58889256/58889256 [==============================] - 4s 0us/step


In [ ]:
last_layer = pre_trained_model.get_layer('block5_pool')
last_output = last_layer.output
x = tf.keras.layers.GlobalMaxPooling2D()(last_output)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(2, activation='sigmoid')(x)


In [ ]:
model = tf.keras.Model(pre_trained_model.input, x)


In [ ]:
model.compile(optimizer='adam', loss=tf.keras.losses.sparse_categorical_crossentropy, metrics=['acc'])


In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
vgg_classifier = model.fit(train_data_gen,
steps_per_epoch=(total_train//batch_size),
epochs = 15,
validation_data=val_data_gen,
validation_steps=(total_val//batch_size),
batch_size = batch_size,
verbose = 1)


Epoch 1/15
103/103 [==============================] - 517s 5s/step - loss: 0.7425 - acc: 0.5109 - val_loss: 0.6982 - val_acc: 0.4569
Epoch 2/15
103/103 [==============================] - 191s 2s/step - loss: 0.6868 - acc: 0.5386 - val_loss: 0.6615 - val_acc: 0.4871
Epoch 3/15
103/103 [==============================] - 70s 679ms/step - loss: 0.6222 - acc: 0.6529 - val_loss: 0.4449 - val_acc: 0.8017
Epoch 4/15
103/103 [==============================] - 41s 405ms/step - loss: 0.4038 - acc: 0.8216 - val_loss: 0.2782 - val_acc: 0.8836
Epoch 5/15
103/103 [==============================] - 9s 86ms/step - loss: 0.3228 - acc: 0.8677 - val_loss: 0.2565 - val_acc: 0.9095
Epoch 6/15
103/103 [==============================] - 6s 60ms/step - loss: 0.3156 - acc: 0.8726 - val_loss: 0.2364 - val_acc: 0.9138
Epoch 7/15
103/103 [==============================] - 5s 53ms/step - loss: 0.3780 - acc: 0.8592 - val_loss: 0.2622 - val_acc: 0.8966
Epoch 8/15
103/103 [==============================] - 4s 39ms/ste

In [ ]:
result = model.evaluate(test_data_gen,batch_size=4)
print("test_loss, test accuracy",result)


25/25 [==============================] - 157s 7s/step - loss: 0.2225 - acc: 0.9031
test_loss, test accuracy [0.22252042591571808, 0.9030612111091614]
